## Zero-shot seniority prediction (OpenRouter)

**Model**: `openai/gpt-5`

This notebook runs **zero-shot** classification on a **stratified 20% test split** from `cleaned_resumes.csv`.

- **Input**: the same feature columns used in your baseline (summary/experience/education/skills/projects/certs + experience time + job title)
- **Output per row**: exactly one word: `junior` / `mid` / `senior`
- **Metrics**: classification report + confusion matrix



### Imports

This cell imports the minimal libraries needed for:
- loading/splitting the data
- calling OpenRouter
- evaluating + plotting results



In [1]:
import os
import json
import re
import time
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import urllib.request
import urllib.error

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay


### Config

This cell sets:
- the model name
- where the CSV is
- OpenRouter settings (API key is read from `OPENROUTER_API_KEY`)
- batch/concurrency knobs (to avoid waiting forever)



In [ ]:
# Model to evaluate (OpenRouter model id)
MODEL = "openai/gpt-5"

# Data (expected to be in the same folder as this notebook)
DATA_PATH = Path("cleaned_resumes.csv")
TARGET = "experience_level"
LABELS = ["junior", "mid", "senior"]

# Use the same baseline feature columns for the LLM prompt
FEATURE_COLS = [
    "summary",
    "experience",
    "education",
    "skills",
    "projects",
    "certifications",
    "last_experience_only",
    "total_experience_time",
    "last_experience_time",
    "job title",
]

# Exclude obvious IDs/metadata (baseline-like)
ID_COLS = {"name", "email", "linkedin", "github", "summary_count", "target_experience_text"}

# OpenRouter
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"

if not OPENROUTER_API_KEY:
    raise ValueError("Missing env var OPENROUTER_API_KEY")

# Speed knobs
BATCH_SIZE = 20          # how many resumes we process at a time
MAX_WORKERS = 8          # parallel requests inside each batch
MAX_RETRIES = 4          # retry on transient API errors
REQUEST_TIMEOUT_S = 90

# If the model returns something invalid after retries, we use this fallback
FALLBACK_LABEL = "mid"


### Load data + build the stratified test split (20%)

This mirrors your baseline setup:
- read `cleaned_resumes.csv`
- auto-pick feature columns (exclude IDs + target)
- create a **stratified** 20% test split with `random_state=42`



In [ ]:
df = pd.read_csv(DATA_PATH)
print("Dataset shape:", df.shape)

if len(df) != 2100:
    raise ValueError(f"Expected 2100 rows, got {len(df)}")

if TARGET not in df.columns:
    raise ValueError(f"Missing target column: {TARGET}")

# Normalize labels
df[TARGET] = df[TARGET].astype(str).str.strip().str.lower()

bad_labels = sorted(set(df[TARGET].unique()) - set(LABELS))
if bad_labels:
    raise ValueError(f"Unexpected labels in {TARGET}: {bad_labels} (expected only {LABELS})")

missing_features = [c for c in FEATURE_COLS if c not in df.columns]
if missing_features:
    raise ValueError(f"Missing required feature columns: {missing_features}")

# Make sure features are strings (safe for prompt building)
for col in FEATURE_COLS:
    df[col] = df[col].astype(str).fillna("")

# Stratified 20% test split (no training used)
_, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df[TARGET]
)

test_df = test_df.reset_index(drop=True)
print("Test rows:", len(test_df))
print(test_df[TARGET].value_counts())

print("Prompt feature columns:", FEATURE_COLS)


Dataset shape: (2100, 17)
Selected feature columns: ['summary', 'experience', 'education', 'skills', 'projects', 'certifications', 'summary_count', 'last_experience_only', 'total_experience_time', 'last_experience_time', 'job title', 'target_experience_text']
Test rows: 420
experience_level
mid       140
junior    140
senior    140
Name: count, dtype: int64


### Prompt + output parsing

We build a single prompt per resume and force the model to answer with **one word only**: `junior`, `mid`, or `senior`.

Then we parse the returned text and normalize it to one of those 3 labels.



In [ ]:
SYSTEM_MSG = (
    "You are a strict classifier. "
    "Return ONLY one word: junior, mid, or senior. "
    "No punctuation. No extra text."
)

def build_user_msg(row: pd.Series) -> str:
    parts = []
    for col in FEATURE_COLS:
        val = str(row.get(col, "")).strip()
        if val:
            parts.append(f"{col}: {val}")
    resume_text = "\n".join(parts)

    return (
        "Classify the candidate's overall seniority into exactly one of: junior, mid, senior.\n"
        "Answer with ONE word only.\n\n"
        f"{resume_text}\n\n"
        "Answer:"
    )

_label_re = re.compile(r"\b(junior|mid|senior)\b", re.IGNORECASE)

def parse_label(text: str) -> str:
    if not text:
        return FALLBACK_LABEL

    t = text.strip().lower()
    # common cleanup
    t = t.replace(".", " ").replace(",", " ").replace("\n", " ")

    # if the first token is a label, use it
    first = t.split()[0] if t.split() else ""
    if first in LABELS:
        return first

    # otherwise try to find any label in the output
    m = _label_re.search(t)
    if m:
        return m.group(1).lower()

    return FALLBACK_LABEL
